# Longitudinal cross sections of the South American Walker circulation and d18ov and specific humidity for the DJF seasonal climatology and difference plots for the warm -- cold phase of IPO, ENSO.

Figure 6. Longitudinal cross-sections of the regional Walker circulation (zonal wind and vertical motion) averaged between 5°S–10°N and oxygen isotopes of water vapor (δ¹⁸Oᵥ) as simulated by iCAM5 during DJF. a) Seasonal climatology for period of study (DJF; 1880 – 2000 CE), b-c) difference between the positive minus negative phase composites of the (b) IPO and (c) ENSO modes. d-e-f) are the same as a-b-c) but showing specific humidity (q). The cross-section spans the eastern Pacific, South America (maximum model topography illustrated with gray shading), and western Atlantic. In b-c) and e-f) only statistically significant vectors (either u or ω, or both) and color shading are plotted (p<0.05).

# Preamble

In [1]:
# ---------------------------- LIBRARIES
import os.path
import numpy as np
import pandas as pd
import xarray as xr   # use xarray to open data files instead of netCDF4.  It is faster, and easier. And, it keeps my metadata!
import netCDF4 as ncd
from netCDF4 import Dataset
import scipy
from scipy.stats import linregress
import metpy.calc as mpcalc
from metpy.units import units

import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import calendar
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import dask.array as da
from shapely.geometry.polygon import LinearRing

import proplot as pplt
import cartopy.feature as cf
from matplotlib import colors
# %matplotlib inline   # This works in jupyter notebooks, otherwise I can't use it. Also, I would need plt.show()

# ---------------------------- Plotting specs
plt.rc('font', family='sans-serif')
plt.rcParams.update({'font.sans-serif':'Arial'})
plt.rcParams.update({'font.size': 12})

# ---------------------------- LOADING FUNCTIONS
def bounds(scalar_fields):
    """
    Get the bounds of a set of scalar_fields
    :param scalar_fields : the scalar field set
    :return: a set of normalized vector field components
    """
    max_bound = -np.inf
    min_bound = np.inf

    for scalar_field in scalar_fields:
        max_lim = np.max(scalar_field)
        min_lim = np.min(scalar_field)
        if max_lim > max_bound:
            max_bound = max_lim
        if min_lim < min_bound:
            min_bound = min_lim

    return np.around(min_bound,2), np.around(max_bound, 2)

def non_time_coords(ds):
    return [v for v in ds.data_vars
            if 'time' not in ds[v].dims]

def drop_non_essential_vars_pop(ds):
    return ds.drop(non_time_coords(ds))  

def drop_all_coords(ds):
    return ds.reset_coords(drop=True)

def geo_loc(target,array):
    return array[np.abs(array - target).argmin()]
# xr.open_mfdataset('*.nc', combine='by_coords', preprocess=drop_all_coords)

def zscr(self):
    return ((self - np.mean(self))/np.std(self))

def detrend_dim(da, dim, deg=1):
    """
    detrend along a single dimension
    """ 
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - (fit - fit.mean())

# Load data

In [2]:
# icam5
path = "/network/rit/home/ro553136/orrison/data/CESM/iCAM5/cntl/atm/f09/"
fh_w = xr.open_dataset(path + "f.e13.FiPIPDC5CN.f09_f09.ctrl.cam.h0.w_1850_2013.nc")
w_reg = fh_w.w.sel(lat=slice(-5,10),lon=slice(260,350))
w_lonP = w_reg[:,12:,:,:].mean('lat')

fh_u = xr.open_dataset(path + "f.e13.FiPIPDC5CN.f09_f09.ctrl.cam.h0.u_1850_2013.nc")
u_reg = fh_u.u.sel(lat=slice(-5,10),lon=slice(260,350))
u_lonP = u_reg[:,12:,:,:].mean('lat')

fh_d18v = xr.open_dataset(path + "f.e13.FiPIPDC5CN.f09_f09.ctrl.cam.h0.d18ov_1850_2013.nc")
d18v_reg = fh_d18v.d18ov.sel(lat=slice(-5,10),lon=slice(260,350))
d18v_lonP = d18v_reg[:,12:,:,:].mean('lat')

fh_q = xr.open_dataset(path + "f.e13.FiPIPDC5CN.f09_f09.ctrl.cam.h0.Q_1850_2013.nc")
q_reg = fh_q.q.sel(lat=slice(-5,10),lon=slice(260,350))
q_lonP = q_reg[:,12:,:,:].mean('lat')
q_lonP = 1000 * q_lonP # Convert from kg/kg to g/kg
lon = fh_w.lon.sel(lon=slice(260,350))
lev = fh_w.lev[12:]

nlon = len(lon)
nlev = len(lev)

# -------------- SUBSET MODEL DATA
icam_yrs = np.arange(1880,2000)
icam_wlonP_djf = w_lonP[30:-14,:,:] 
icam_ulonP_djf = u_lonP[30:-14,:,:] 
icam_d18vlonP_djf = d18v_lonP[30:-14,:,:] 
icam_qlonP_djf = q_lonP[30:-14,:,:] 

# -------------------------- detrend model 
w_dt = np.zeros(icam_wlonP_djf.shape)
u_dt = np.zeros(icam_ulonP_djf.shape)
d18ov_dt = np.zeros(icam_d18vlonP_djf.shape)
q_dt = np.zeros(icam_qlonP_djf.shape)
for x in range(len(lev)):
    for y in range(len(lon)):
        w_dt[:,x,y] = detrend_dim(icam_wlonP_djf[:,x,y],'time',1)
        u_dt[:,x,y] = detrend_dim(icam_ulonP_djf[:,x,y],'time',1)
        d18ov_dt[:,x,y] = detrend_dim(icam_d18vlonP_djf[:,x,y],'time',1)
        q_dt[:,x,y] = detrend_dim(icam_qlonP_djf[:,x,y],'time',1)

# -------------------------- Format as xr
w_dt = xr.DataArray(w_dt, 
    coords={'time': icam_yrs,'lev': lev,'lon': lon}, 
    dims=["time","lev", "lon"])
w_comp_dt = np.mean(w_dt, axis=0)

u_dt = xr.DataArray(u_dt, 
    coords={'time': icam_yrs,'lev': lev,'lon': lon}, 
    dims=["time","lev", "lon"])
u_comp_dt = np.mean(u_dt, axis=0)

d18ov_dt = xr.DataArray(d18ov_dt, 
    coords={'time': icam_yrs,'lev': lev,'lon': lon}, 
    dims=["time","lev", "lon"])
d18ov_comp_dt = np.mean(d18ov_dt, axis=0)

q_dt = xr.DataArray(q_dt, 
    coords={'time': icam_yrs,'lev': lev,'lon': lon}, 
    dims=["time","lev", "lon"])
q_comp_dt = np.mean(q_dt, axis=0)

In [3]:
# LOAD topography
datapath = "/network/rit/home/ro553136/orrison/data/CESM/iCAM5/cntl/atm/f09/"
fname = "fv0.9x1.25-default-PHIS-new-SGH-SGH30.nc"

#  Notice that in Python we can easily concatenate strings together just by `adding` them
fh = xr.open_dataset(datapath + fname)

height_raw = fh.PHIS.sel(lat=slice(-50,10),lon=slice(260,350)) 
height = height_raw / 9.8 # in meters

# height_xsec_avg = np.mean(height, 0)
height_xsec_max = np.max(height, 0)
# height_xsec_med = np.median(height, 0)
# height_xsec_equa = height.sel(lat=0, method='nearest')

height_xsec_max = height_xsec_max * units.meters
height_xsec_max_hPa = mpcalc.height_to_pressure_std(height_xsec_max)
surface = [1013.25]*73

# IPO

In [4]:
# -------------------------- LOAD IPO TIME SERIES
pth_ipo = '/network/rit/home/ro553136/orrison/data/reconstructions/sstforc_Had_OI/'
fh_ipo = xr.open_dataset(pth_ipo + 'HADSSTOI_ipoinds_1880_2000.nc', decode_times= True)
ipo_index = fh_ipo.tpi_index_11yr
pw = []
pc = []
for i in range(len(ipo_index)):
    if ipo_index[i] >= np.mean(ipo_index) + np.std(ipo_index)/2:
        pw.append(float(ipo_index.time[i]))
    elif ipo_index[i] <= np.mean(ipo_index) - np.std(ipo_index)/2:
        pc.append(float(ipo_index.time[i]))
    else:
        continue

### Composite detrended model data

In [5]:
# -------------------------- w: COMPOSITE detrended MODEL data, detrended index
w_pw_ipo_dt_all = xr.concat([w_dt.sel(time=i, method='nearest') for i in pw],dim='time')
w_pc_ipo_dt_all = xr.concat([w_dt.sel(time=i, method='nearest') for i in pc],dim='time')
w_pw_ipo_dt = np.mean(w_pw_ipo_dt_all, axis=0)
w_pc_ipo_dt = np.mean(w_pc_ipo_dt_all, axis=0)

# -------------------------- u: COMPOSITE detrended MODEL data, detrended index
u_pw_ipo_dt_all = xr.concat([u_dt.sel(time=i, method='nearest') for i in pw],dim='time')
u_pc_ipo_dt_all = xr.concat([u_dt.sel(time=i, method='nearest') for i in pc],dim='time')
u_pw_ipo_dt = np.mean(u_pw_ipo_dt_all, axis=0)
u_pc_ipo_dt = np.mean(u_pc_ipo_dt_all, axis=0)

# -------------------------- d18ov: COMPOSITE detrended MODEL data, detrended index
d18ov_pw_ipo_dt_all = xr.concat([d18ov_dt.sel(time=i, method='nearest') for i in pw],dim='time')
d18ov_pc_ipo_dt_all = xr.concat([d18ov_dt.sel(time=i, method='nearest') for i in pc],dim='time')
d18ov_pw_ipo_dt = np.mean(d18ov_pw_ipo_dt_all, axis=0)
d18ov_pc_ipo_dt = np.mean(d18ov_pc_ipo_dt_all, axis=0)

# -------------------------- q: COMPOSITE detrended MODEL data, detrended index
q_pw_ipo_dt_all = xr.concat([q_dt.sel(time=i, method='nearest') for i in pw],dim='time')
q_pc_ipo_dt_all = xr.concat([q_dt.sel(time=i, method='nearest') for i in pc],dim='time')
q_pw_ipo_dt = np.mean(q_pw_ipo_dt_all, axis=0)
q_pc_ipo_dt = np.mean(q_pc_ipo_dt_all, axis=0)

### stat sig

In [6]:
    # Arrays for the warm - cold phase
om_d_ipo_t, om_d_ipo_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
u_d_ipo_t, u_d_ipo_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
d18v_d_ipo_t, d18v_d_ipo_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
q_d_ipo_t, q_d_ipo_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 

    for x in range(len(lev)):
        for y in range(len(lon)):
            om_d_ipo_t[x,y],om_d_ipo_p[x,y] = scipy.stats.ttest_ind(w_pw_ipo_dt_all[:,x,y],w_pc_ipo_dt_all[:,x,y],equal_var=False)
            u_d_ipo_t[x,y],u_d_ipo_p[x,y] = scipy.stats.ttest_ind(u_pw_ipo_dt_all[:,x,y],u_pc_ipo_dt_all[:,x,y],equal_var=False)
            d18v_d_ipo_t[x,y],d18v_d_ipo_p[x,y] = scipy.stats.ttest_ind(d18ov_pw_ipo_dt_all[:,x,y],d18ov_pc_ipo_dt_all[:,x,y],equal_var=False)        
            q_d_ipo_t[x,y],q_d_ipo_p[x,y] = scipy.stats.ttest_ind(q_pw_ipo_dt_all[:,x,y],q_pc_ipo_dt_all[:,x,y],equal_var=False)        

In [7]:
    # Walker vectors: positive - negative phase
# om_d_ipo
[m,n] = np.where(om_d_ipo_p < 0.1)  # make a 2-d array of grid cells where p is significant
om_d_ipo_hatch=np.ones(om_d_ipo_t.shape)
om_d_ipo_hatch[m, n] = 0

# u_d_ipo
[m,n] = np.where(u_d_ipo_p < 0.1)  # make a 2-d array of grid cells where u is significant
u_d_ipo_hatch=np.ones(u_d_ipo_t.shape)
u_d_ipo_hatch[m, n] = 0

# # om_d_ipo + u_d_ipo
merge = om_d_ipo_hatch+u_d_ipo_hatch
omu_d_ipo_hatch = np.where(merge < 2,0,1)

# d18_d_ipo
[m,n] = np.where(d18v_d_ipo_p < 0.1)  # make a 2-d array of grid cells where u is significant
d18v_d_ipo_hatch=np.ones(d18v_d_ipo_t.shape)
d18v_d_ipo_hatch[m, n] = 0

# q_d_ipo
[m,n] = np.where(q_d_ipo_p < 0.1)  # make a 2-d array of grid cells where u is significant
q_d_ipo_hatch=np.ones(q_d_ipo_t.shape)
q_d_ipo_hatch[m, n] = 0

    # Mask out non-sig arrays
#positive - negative phase
om_d_ipo_anom = w_pw_ipo_dt - w_pc_ipo_dt
u_d_ipo_anom = u_pw_ipo_dt - u_pc_ipo_dt
d18v_d_ipo_anom = d18ov_pw_ipo_dt-d18ov_pc_ipo_dt

om_d_ipo_anom_m = np.ma.masked_array(om_d_ipo_anom, mask = omu_d_ipo_hatch)
u_d_ipo_anom_m = np.ma.masked_array(u_d_ipo_anom, mask = omu_d_ipo_hatch)
d18v_d_ipo_anom_m = np.ma.masked_array(d18v_d_ipo_anom, mask = d18v_d_ipo_hatch)

q_d_ipo_anom = q_pw_ipo_dt-q_pc_ipo_dt
q_d_ipo_anom_m = np.ma.masked_array(q_d_ipo_anom, mask = q_d_ipo_hatch)


# ENSO

In [8]:
# -------------------------- LOAD, DETREND N34 TIME SERIES
pth_enso = '/network/rit/home/ro553136/orrison/data/reconstructions/sstforc_Had_OI/'
fh_ens = xr.open_dataset(pth_enso + "HADOISST_N34_dt_1880_2000.nc") # Try with detrending

n34ind_had = fh_ens.n34ind_had

# calculate DJF of indices
n34ind_djf = [np.nanmean(n34ind_had[11+i:14+i],axis=0) for i in range(0,1440,12)]
    # select period only for 1880 -- 1999
n34ind_djf_had_xr = xr.DataArray(n34ind_djf, 
    coords={'time': icam_yrs}, 
    dims=["time"])

In [9]:
# -------------------------- SUBSET INTO WARM AND COLD N34 YEARS. From d18op composite script
pw = []
pc = []

# -------------------------- +/- 0.5 threshold
for i in range(len(n34ind_djf_had_xr)):
    if n34ind_djf_had_xr[i] >= 0.5:
        pw.append(float(n34ind_djf_had_xr.time[i]))
    elif n34ind_djf_had_xr[i] <= -0.5:
        pc.append(float(n34ind_djf_had_xr.time[i]))
    else:
        continue        

# -------------------------- w: COMPOSITE detrended MODEL data, detrended index
w_pw_enso_dt_all = xr.concat([w_dt.sel(time=i, method='nearest') for i in pw],dim='time')
w_pc_enso_dt_all = xr.concat([w_dt.sel(time=i, method='nearest') for i in pc],dim='time')
w_pw_enso_dt = np.mean(w_pw_enso_dt_all, axis=0)
w_pc_enso_dt = np.mean(w_pc_enso_dt_all, axis=0)

# -------------------------- u: COMPOSITE detrended MODEL data, detrended index
u_pw_enso_dt_all = xr.concat([u_dt.sel(time=i, method='nearest') for i in pw],dim='time')
u_pc_enso_dt_all = xr.concat([u_dt.sel(time=i, method='nearest') for i in pc],dim='time')
u_pw_enso_dt = np.mean(u_pw_enso_dt_all, axis=0)
u_pc_enso_dt = np.mean(u_pc_enso_dt_all, axis=0)

# -------------------------- d18ov: COMPOSITE detrended MODEL data, detrended index
d18ov_pw_enso_dt_all = xr.concat([d18ov_dt.sel(time=i, method='nearest') for i in pw],dim='time')
d18ov_pc_enso_dt_all = xr.concat([d18ov_dt.sel(time=i, method='nearest') for i in pc],dim='time')
d18ov_pw_enso_dt = np.mean(d18ov_pw_enso_dt_all, axis=0)
d18ov_pc_enso_dt = np.mean(d18ov_pc_enso_dt_all, axis=0)

# -------------------------- q: COMPOSITE detrended MODEL data, detrended index
q_pw_enso_dt_all = xr.concat([q_dt.sel(time=i, method='nearest') for i in pw],dim='time')
q_pc_enso_dt_all = xr.concat([q_dt.sel(time=i, method='nearest') for i in pc],dim='time')
q_pw_enso_dt = np.mean(q_pw_enso_dt_all, axis=0)
q_pc_enso_dt = np.mean(q_pc_enso_dt_all, axis=0)

# Statistical significance
For anomalies - the w/c period relative to the neutral years 

In [10]:
    # Arrays for the warm - cold phase
om_d_n34_t, om_d_n34_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
u_d_n34_t, u_d_n34_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
d18v_d_n34_t, d18v_d_n34_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 
q_d_n34_t, q_d_n34_p = (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])), (xr.DataArray(np.zeros([len(lev), len(lon)]), coords={"lev":lev,"lon":lon}, dims=["lev","lon"])) 

    for x in range(len(lev)):
        for y in range(len(lon)):
            om_d_n34_t[x,y],om_d_n34_p[x,y] = scipy.stats.ttest_ind(w_pw_enso_dt_all[:,x,y],w_pc_enso_dt_all[:,x,y],equal_var=False)
            u_d_n34_t[x,y],u_d_n34_p[x,y] = scipy.stats.ttest_ind(u_pw_enso_dt_all[:,x,y],u_pc_enso_dt_all[:,x,y],equal_var=False)
            d18v_d_n34_t[x,y],d18v_d_n34_p[x,y] = scipy.stats.ttest_ind(d18ov_pw_enso_dt_all[:,x,y],d18ov_pc_enso_dt_all[:,x,y],equal_var=False)        
            q_d_n34_t[x,y],q_d_n34_p[x,y] = scipy.stats.ttest_ind(q_pw_enso_dt_all[:,x,y],q_pc_enso_dt_all[:,x,y],equal_var=False)        

In [11]:
# -------------------------- Walker vectors: positive - negative phase
# om_n
[m,n] = np.where(om_d_n34_p < 0.1)  # make a 2-d array of grid cells where p is significant
om_d_n34_hatch=np.ones(om_d_n34_t.shape)
om_d_n34_hatch[m, n] = 0

# u_n
[m,n] = np.where(u_d_n34_p < 0.1)  # make a 2-d array of grid cells where u is significant
u_d_n34_hatch=np.ones(u_d_n34_t.shape)
u_d_n34_hatch[m, n] = 0

# # om_n + u_n
merge = om_d_n34_hatch+u_d_n34_hatch
omu_d_n34_hatch = np.where(merge < 2,0,1)

#  d18v pos, neg, d_n34
[m,n] = np.where(d18v_d_n34_p < 0.1)  # make a 2-d array of grid cells where u is significant
d18v_d_n34_hatch=np.ones(d18v_d_n34_t.shape)
d18v_d_n34_hatch[m, n] = 0

#  q pos, neg, d_n34
[m,n] = np.where(q_d_n34_p < 0.05)  # make a 2-d array of grid cells where q is significant
q_d_n34_hatch=np.ones(q_d_n34_t.shape)
q_d_n34_hatch[m, n] = 0

# Mask arrays for only significant vectors, plot these. 
#positive - negative phase
om_d_n34_anom = w_pw_enso_dt - w_pc_enso_dt
u_d_n34_anom = u_pw_enso_dt - u_pc_enso_dt
d18v_d_n34_anom = d18ov_pw_enso_dt-d18ov_pc_enso_dt
q_d_n34_anom = q_pw_enso_dt-q_pc_enso_dt

om_d_n34_anom_m = np.ma.masked_array(om_d_n34_anom, mask = omu_d_n34_hatch)
u_d_n34_anom_m = np.ma.masked_array(u_d_n34_anom, mask = omu_d_n34_hatch)
d18v_d_n34_anom_m = np.ma.masked_array(d18v_d_n34_anom, mask = d18v_d_n34_hatch)
q_d_n34_anom_m = np.ma.masked_array(q_d_n34_anom, mask = q_d_n34_hatch)

# Plotting

In [12]:
# Create a figure and three subplots
fig = plt.figure(figsize=(14, 10), dpi=300)
gs = GridSpec(2, 5, width_ratios=[1, 0.08, 1, 1, 0.08])
gs.update(hspace=0.3)
# Adjust the layout to make room for labels
fig.subplots_adjust(wspace=0.5)
ax1 = fig.add_subplot(gs[0,0])
ax2 = fig.add_subplot(gs[0,1])
ax3 = fig.add_subplot(gs[0,2])
ax4 = fig.add_subplot(gs[0,3])
ax5 = fig.add_subplot(gs[0,4])

ax6 = fig.add_subplot(gs[1,0])
ax7 = fig.add_subplot(gs[1,1])
ax8 = fig.add_subplot(gs[1,2])
ax9 = fig.add_subplot(gs[1,3])
ax10 = fig.add_subplot(gs[1,4])

    # ------
    # Climatology
    # ------   
        # d18ov contours
levs_comp = np.arange(-100.0, 0.0,5)
climo = ax1.contourf(lon, lev, d18ov_comp_dt, cmap='viridis', levels = levs_comp, extend='both')
# ax1.autoscale(False)

        # vectors, scale w for visibility
skip = 6
walk_comp = ax1.quiver(lon[::skip], lev, u_comp_dt[:,::skip], w_comp_dt[:,::skip]*-250,
                                scale=100, width=0.006, headwidth=3., headlength=4.)
qk = plt.quiverkey(walk_comp, 0.335, 0.6, 5.0,  r'$5 \,\, m \dot s^{-1},$''\n'r'$ \,\,\, -250 \,\, Pa \dot s^{-1}$', labelsep=0.1, 
                   labelpos='S', coordinates='figure', fontproperties={'size': 10}) #r'$5 \,\, \frac{m}{s}$'

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Height (hPa)')
ax1.set_title('Climatology', fontsize=16, weight='bold')

pos = ax2.get_position()
new_pos = [pos.x0-.04, pos.y0+0.075, pos.width, pos.height-0.1]
ax2.set_position(new_pos)
fig.colorbar(climo, cax=ax2, orientation='vertical', label='permil') 

    # ------
    # IPO
    # ------  
# Plot omega anomaly contours
levs_comp = np.arange(-2.0, 5.4,0.4)
divnorm = colors.TwoSlopeNorm(vmin=-2., vcenter=0., vmax=5)

ipo = ax3.contourf(lon, lev, d18v_d_ipo_anom_m, cmap='brbg_r', norm=divnorm, levels = levs_comp, extend='both')

# Plot Walker circulation quivers (x, y, U, Omega)
skip = 6
walk_ipo = ax3.quiver(lon[::skip], lev, u_d_ipo_anom_m[:,::skip], om_d_ipo_anom_m[:,::skip]*-250, 
                                scale=30, width=0.006, color='black', headwidth=3., headlength=4.)
ax3.set_xlabel('Longitude')
ax3.set_title('IPO', fontsize=16, weight='bold')

    # ------
    # ENSO
    # ------ 
# plot sig d18ov d_n34 anomaly contours
om_n34 = ax4.contourf(lon, lev, d18v_d_n34_anom_m, cmap='brbg_r', norm=divnorm, levels = levs_comp, extend='both')
# axs[2].autoscale(False)

# Plot Walker circulation quivers (x, y, U, Omega)
skip = 6
walk_n34 = ax4.quiver(lon[::skip], lev, u_d_n34_anom_m[:,::skip], om_d_n34_anom_m[:,::skip]*-250, 
                                scale=30, width=0.006, color='black', headwidth=3., headlength=4.)
qk = plt.quiverkey(walk_n34, 0.865, 0.6, 3.0,  r'$3 \,\, m \dot s^{-1},$''\n'r'$ \,\,\, -250 \,\, Pa \dot s^{-1}$', 
                   labelsep=0.1, labelpos='S', coordinates='figure', fontproperties={'size': 10}) 
ax4.set_xlabel('Longitude')
ax4.set_title('ENSO', fontsize=16, weight='bold')

pos = ax5.get_position()
new_pos = [pos.x0-.04, pos.y0+0.075, pos.width, pos.height-0.1]
ax5.set_position(new_pos)
fig.colorbar(om_n34, cax=ax5, orientation='vertical', label='permil anomalies')

    #########################################
    
    # ------
    # Climatology
    # ------   
        # q contours
levs_comp_q = np.arange(0, 17.5,1.5)
climo = ax6.contourf(lon, lev, q_comp_dt, cmap='Greens', levels = levs_comp_q, extend='both')
# ax1.autoscale(False)

        # vectors, scale w for visibility
skip = 6
walk_comp = ax6.quiver(lon[::skip], lev, u_comp_dt[:,::skip], w_comp_dt[:,::skip]*-250,
                                scale=100, width=0.006, headwidth=3., headlength=4.)
qk = plt.quiverkey(walk_comp, 0.335, 0.16, 5.0,  r'$5 \,\, m \dot s^{-1},$''\n'r'$ \,\,\, -250 \,\, Pa \dot s^{-1}$', labelsep=0.1, 
                   labelpos='S', coordinates='figure', fontproperties={'size': 10}) #r'$5 \,\, \frac{m}{s}$'

ax6.set_xlabel('Longitude')
ax6.set_ylabel('Height (hPa)')
ax6.set_title('Climatology', fontsize=16, weight='bold')

pos = ax7.get_position()
new_pos = [pos.x0-.04, pos.y0+0.075, pos.width, pos.height-0.1]
ax7.set_position(new_pos)
fig.colorbar(climo, cax=ax7, orientation='vertical', label='q (g/kg)')

    # ------
    # IPO
    # ------  
# Plot omega anomaly contours
levs_comp_q = np.arange(-0.2, 0.2, 0.04)
# divnorm = colors.TwoSlopeNorm(vmin=-2., vcenter=0., vmax=5)
ipo = ax8.contourf(lon, lev, q_d_ipo_anom_m, cmap='PiYG', levels = levs_comp_q, extend='both') #norm=divnorm, 

# Plot Walker circulation quivers (x, y, U, Omega)
skip = 6
walk_ipo = ax8.quiver(lon[::skip], lev, u_d_ipo_anom_m[:,::skip], om_d_ipo_anom_m[:,::skip]*-250, 
                                scale=30, width=0.006, color='black', headwidth=3., headlength=4.)
ax8.set_xlabel('Longitude')
ax8.set_title('IPO', fontsize=16, weight='bold')

    # ------
    # ENSO
    # ------ 
# plot sig d18ov d_n34 anomaly contours
om_n34 = ax9.contourf(lon, lev, q_d_n34_anom_m, cmap='PiYG', levels = levs_comp_q, extend='both') #norm=divnorm, 
# axs[2].autoscale(False)

# Plot Walker circulation quivers (x, y, U, Omega)
skip = 6
walk_n34 = ax9.quiver(lon[::skip], lev, u_d_n34_anom_m[:,::skip], om_d_n34_anom_m[:,::skip]*-250, 
                                scale=30, width=0.006, color='black', headwidth=3., headlength=4.)
qk = plt.quiverkey(walk_n34, 0.865, 0.16, 3.0,  r'$3 \,\, m \dot s^{-1},$''\n'r'$ \,\,\, -250 \,\, Pa \dot s^{-1}$', 
                   labelsep=0.1, labelpos='S', coordinates='figure', fontproperties={'size': 10}) 
ax9.set_xlabel('Longitude')
ax9.set_title('ENSO', fontsize=16, weight='bold')

pos = ax10.get_position()
new_pos = [pos.x0-.04, pos.y0+0.075, pos.width, pos.height-0.1]
ax10.set_position(new_pos)
fig.colorbar(om_n34, cax=ax10, orientation='vertical', label='q anomalies (g/kg)')

    # ------
    # Set options 
    # ------     
lets = ['a','b','c', 'd', 'e','f']
for i,ax in enumerate([ax1,ax3,ax4,ax6,ax8,ax9]):
    ax.invert_yaxis()
    ax.set_yscale("log")
    ax.set_yticks([1000,925,850,700,500,300,200])
    ax.set_yticklabels([1000,925,850,700,500,300,200])
    ax.set_ylim(1000, 200)

    # Add South American max topography
    ax.fill_between(x=height_xsec_max_hPa.lon, y1=height_xsec_max_hPa, y2=surface, color='grey',  interpolate=True, alpha=1)

    # Format longitudes
    lon_fmt = LongitudeFormatter(number_format='.0f')
    ax.xaxis.set_major_formatter(lon_fmt)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(30))

    # Label panels
    ax.text(0.05,1.05,lets[i], weight='bold', fontsize='16',
            horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

# Display the plot
# plt.tight_layout()
# plt.savefig('/network/rit/home/ro553136/orrison/Plots/Pac_SAm/r2r/Fig6_Walker_d18ov_q_diff_ipo_n34_10N_5S_SAm_topo.jpg',format='JPEG',dpi=300)
plt.show()

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Font family ['sans'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans' not found because none of the following families were found: Arial
findfont: Font family ['sans'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans' not found because none of the following families were found: Arial
findfont: Font family ['sans'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans' not found b